In [ ]:
import pandas as pd
import gc

train_df = pd.read_parquet('/storage/fp1_features_no_norm/VKS/public_train.pqt')
test_df = pd.read_parquet('/storage/fp1_features_no_norm/VKS/submit_2.pqt')
q = pd.read_parquet('/storage/fp1_features_no_norm/VKS/competition_data_final_pqt')
df_test_urls = q.loc[q.user_id.isin(test_df.user_id.values.tolist())].url_host.unique().tolist()

print(q.shape)
q = q.loc[q.url_host.isin(df_test_urls)].reset_index(drop=True)
print(q.shape)

del df_test_urls
gc.collect()

q['date'] = pd.to_datetime(q['date'])
q['mobile'] = q['url_host'].str.startswith('m.')
q.loc[q.mobile, 'url_host'] = q.loc[q.mobile, 'url_host'].apply(lambda x: x[2:])
q['url_host'] = q['url_host'].apply(lambda x: x.split('.')[:-1] if x.find('.') >= 0 else [x])
q['url_host'] = q['url_host'].apply(lambda x: '.'.join(x))

train_q = q.loc[q.user_id.isin(train_df.user_id.values.tolist())]
d_train_q = train_q['url_host'].value_counts().to_dict()
del train_q
gc.collect()

q['url_host_freq'] = q['url_host'].apply(lambda x: d_train_q.get(x,0))
del d_train_q
gc.collect()

q.loc[q.url_host_freq<3,'url_host'] = 'unknown'

In [3]:
import pandas as pd
import polars as pl

In [4]:
urls_mapping = pl.read_parquet("final.pq")

In [28]:
urls_mapping["url_cleaned_2"].n_unique()

104959

In [5]:
sessions = pl.read_parquet("../data/processed/sessions.pq").join(urls_mapping, how="left", on="url_host")

In [6]:
%%time
users_urls = (
    sessions
    # .filter(pl.col("url_cleaned_2").is_not_null())
    .groupby("user_id")
    .agg(pl.col("url_cleaned_2").unique().alias("urls"))
)

CPU times: user 50.4 s, sys: 8.09 s, total: 58.5 s
Wall time: 30.3 s


In [9]:
users_urls[:10].with_columns(pl.col("urls").apply(lambda x: list(filter(lambda a: a is not None, x))))

user_id,urls
u32,list[str]
68072,"[""exchange.buzzoola"", ""habr"", ... ""googleads.g.doubleclick""]"
19480,"[""poncy"", ""mexn"", ... ""russian.people.com""]"
51032,"[""vc"", ""lookmeet"", ... ""kinoaction""]"
358608,"[""mag.103"", ""profile"", ... ""ssp.bidvol""]"
383600,"[""megapoisk"", ""avtodispetcher"", ... ""play.google""]"
364504,"[""sun9-27.userapi"", ""text-pesni"", ... ""relap""]"
159848,"[""googleads.g.doubleclick""]"
228632,"[""ssp.otm-r"", ""akkords"", ... ""play.google""]"
54936,"[""ad.mail"", ""s14.stc.all.kpcdn"", ... ""vk""]"


In [15]:
users_urls_ = (
    users_urls
    .explode("urls")
    .filter(pl.col("urls").is_not_null())
    .groupby("user_id")
    .agg(pl.col("urls").unique())
)

In [20]:
%%time
users_urls_text = users_urls_.with_columns(pl.col("urls").apply(lambda x: " ".join(x)))

CPU times: user 54 s, sys: 1.17 s, total: 55.2 s
Wall time: 55.2 s


In [24]:
users_urls_.explode("urls")["urls"].n_unique()

104959

In [23]:
users_urls_text.write_parquet("../data/features/url/urls_text-ultra-filtered.pq")

In [ ]:
users_urls

In [6]:
sessions["url_cleaned_2"].is_null().sum()

149027

In [ ]:
%%time
sessions = sessions

In [5]:
train = pl.read_parquet("../data/processed/train.pq")
test = pl.read_parquet("../data/processed/test.pq")

In [6]:
%%time
test_urls = set(test.join(sessions, how="left", on="user_id")["url_host"].unique().to_list())

CPU times: user 19.9 s, sys: 10.2 s, total: 30.1 s
Wall time: 11 s


In [7]:
len(test_urls)

134338

In [8]:
%%time
train_users = train["user_id"].unique().to_list()

CPU times: user 37.3 ms, sys: 8.38 ms, total: 45.7 ms
Wall time: 16 ms


In [9]:
%%time
sessions = sessions.filter(pl.col("url_host").is_in(list(test_urls)) & pl.col("user_id").is_in(list(train_users)))

CPU times: user 23.3 s, sys: 13 s, total: 36.3 s
Wall time: 19.9 s


In [15]:
def clean_mobile(url: str) -> str:
    if url.startswith("m."):
        return url[2:]
    return url

def remove_first_level(url: str) -> str:
    if url.find(".") >= 0:
        return ".".join(url.split(".")[:-1])
    return url

urls_map = pd.DataFrame()
urls_map["urls"] = list(test_urls)
urls_map["url_cleaned"] = urls_map["urls"].map(clean_mobile).map(remove_first_level)

In [21]:
%%time
sessions = sessions.join(pl.from_pandas(urls_map), how="left", left_on="url_host", right_on="urls")

CPU times: user 30.3 s, sys: 14.8 s, total: 45.1 s
Wall time: 12.2 s


In [23]:
url_stats = sessions["url_cleaned"].value_counts()

In [25]:
url_stats.write_parquet("url_stats.pq")

In [31]:
urls_map_2 = url_stats.with_columns(
    pl.when(pl.col("counts") < 3).then(pl.lit("unknown")).otherwise(pl.col("url_cleaned")).alias("url_cleaned_2")
).select(["url_cleaned", "url_cleaned_2"])

In [32]:
%%time
sessions = sessions.join(urls_map_2, how="left", on="url_cleaned")

CPU times: user 30.4 s, sys: 14.2 s, total: 44.6 s
Wall time: 11.7 s


In [35]:
urls_final_mapping = sessions.select(["url_host", "url_cleaned_2"]).unique()

In [39]:
urls_final_mapping.write_parquet("final.pq")

In [40]:
sessions["url_host"].n_unique()

106786

In [48]:
u = pd.DataFrame()
u["url_host"] = list(set(test_urls) - set(urls_final_mapping["url_host"]))
u["url_cleaned_2"] = u["url_host"].map(clean_mobile).map(remove_first_level)

In [51]:
pl.concat([urls_final_mapping, pl.from_pandas(u)]).write_parquet("final.pq")

In [45]:
len(u) + len(urls_final_mapping)

134338

In [47]:
len(test_urls)

134338

In [ ]:
%%time
sessions = sessions.with_columns(
    pl.when(pl.col("url_host").str.starts_with("m."))
    .then(pl.col("url_host").apply(lambda x: x[2:]))
    .otherwise(pl.col("url_host"))
)

In [20]:
sessions[:1000].filter(pl.col("url_host").str.starts_with("m.")).with_columns(
    pl.when(pl.col("url_host").str.starts_with("m."))
    .then(pl.col("url_host").apply(lambda x: x[2:]))
    .otherwise(pl.col("url_host"))
)

region_id,city_id,manufacturer_id,model_id,url_host,price,date,part_of_day_id,request_cnt,user_id
u8,u16,u8,u16,str,f32,date,u8,u8,u32
61,741,24,396,"""sports.ru""",74259.0,2021-09-16,3,2,79395
61,741,24,396,"""sport-express....",74259.0,2021-09-14,3,1,79395
61,741,24,396,"""lenta.ru""",74259.0,2021-09-15,3,1,79395
61,741,24,396,"""lenta.ru""",74259.0,2021-08-29,3,1,79395
61,741,24,396,"""facebook.com""",74259.0,2021-09-29,2,1,79395
61,741,24,396,"""sports.ru""",74259.0,2021-09-29,3,2,79395
61,741,24,396,"""gazeta.ru""",74259.0,2021-09-15,3,1,79395
61,741,24,396,"""hightech.plus""",74259.0,2021-10-04,3,1,79395
61,741,24,396,"""lenta.ru""",74259.0,2021-10-04,3,1,79395


In [6]:
train.with_columns(pl.lit(True).alias("is_train"))

user_id,is_male,age,age_bucket,is_train
u32,u8,u16,u8,bool
350459,1,31,2,true
188276,1,35,2,true
99002,0,41,3,true
155506,0,33,2,true
213873,0,54,4,true
212300,0,63,5,true
268755,1,33,2,true
350740,1,39,3,true
357123,0,27,2,true


In [3]:
sessions["url_host"]

region_id,city_id,manufacturer_id,model_id,url_host,price,date,part_of_day_id,request_cnt,user_id
u8,u16,u8,u16,str,f32,date,u8,u8,u32
22,255,2,29,"""ad.adriver.ru""",20368.0,2022-06-15,1,1,45098
22,255,2,29,"""apple.com""",20368.0,2022-06-19,1,1,45098
22,255,2,29,"""avatars.mds.ya...",20368.0,2022-06-12,2,1,45098
22,255,2,29,"""googleads.g.do...",20368.0,2022-05-16,2,1,45098
22,255,2,29,"""googleads.g.do...",20368.0,2022-05-30,2,1,45098
22,255,2,29,"""i.ytimg.com""",20368.0,2022-03-29,3,2,45098
22,255,2,29,"""icloud.com""",20368.0,2022-03-17,1,1,45098
22,255,2,29,"""m.avito.ru""",20368.0,2022-05-19,1,1,45098
22,255,2,29,"""relap.io""",20368.0,2022-03-29,4,1,45098
